# All customer 'OrderQuantity'<br>
In this file, all customer order quantities are predicted. Other files were predicting order quantities based on the customer.

In [1]:
%pip install nbformat pandas numpy matplotlib seaborn scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
import sys
sys.path.append('./../src')

In [4]:
# Load the dataset
product_sales_df = pd.read_csv('../datasets/forecasting/final/product_sales.csv')

# Reducing df size due to issues with not enough system hardware for hyperparam training
fraction = 0.30

# Randomly sample the DataFrame
product_sales_df = product_sales_df.sample(frac=fraction, random_state=42)

### Z-score removal
This feature is controversial. In SCM, volitility is inevitable and the model SHOULD be trained to handle this. It can also skew all predictions making all predictions inaccurate, where as if these are removed predictions for OrderQuantity could be more accurate.

__If wanting to keep the abnormally high OrderQuantity values, COMMENT THIS OUT!__

In [5]:
from scipy import stats

# Define a function to compute z-scores within each product group
def compute_zscore(group, threshold=3):
    # Only compute z-score if there are at least 2 data points in the group
    if len(group) >= 2:
        group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
    else:
        group['z_score'] = 0  # or np.nan if preferred
    return group

# Group by "ProductNumber" and compute z-scores for "OrderQuantity" within each group
df_grouped = product_sales_df.groupby('ProductNumber').apply(compute_zscore)

# Define your threshold for what constitutes an "unusually high" order
z_threshold = 3

# Filter for outliers (orders with a z_score greater than the threshold)
df_outliers = df_grouped[df_grouped['z_score'] > z_threshold]

# Display the results
print("Unusually high order quantities (per ProductNumber):")
print(df_outliers[['ProductNumber', 'OrderQuantity', 'z_score']])

print(f'Dropping: \n{df_outliers.count()}')
# Keep only rows where the z-score is within the acceptable range
df_cleaned = df_grouped[df_grouped['z_score'] <= z_threshold].drop(columns=['z_score'])

# Reset index for clarity
df_cleaned.reset_index(drop=True, inplace=True)

product_sales_df = df_cleaned

C:\Users\44777\AppData\Local\Temp\ipykernel_10632\1902335139.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
C:\Users\44777\AppData\Local\Temp\ipykernel_10632\1902335139.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
C:\Users\44777\AppData\Local\Temp\ipykernel_10632\1902335139.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
C:\Users\44777\AppData\Local\Temp\ipykernel_10632\1902335139.py:7: RuntimeWarning: Preci

Unusually high order quantities (per ProductNumber):
                     ProductNumber  OrderQuantity   z_score
ProductNumber                                              
FRE-005-0049Z 11903  FRE-005-0049Z         720000  3.032564
UND-026-0086Z 48616  UND-026-0086Z         720000  3.361733
UND-026-0092Y 48792  UND-026-0092Y         204000  3.085499
Dropping: 
ProductNumber         3
order_year            3
order_month           3
order_week            3
order_weekday         3
is_weekend            3
Customer_Num          3
OrderQuantity         3
prev_month_sales      3
prev_week_sales       3
prev_2_month_sales    3
prev_3_month_sales    3
var_1m                0
var_3m                3
var_6m                3
var_12m               3
var_18m               3
log_var_1m            0
log_var_3m            3
log_var_6m            3
log_var_12m           3
log_var_18m           3
yoy_growth            3
moving_avg_3m         3
moving_avg_6m         3
moving_avg_12m        3
moving_avg_1

C:\Users\44777\AppData\Local\Temp\ipykernel_10632\1902335139.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = product_sales_df.groupby('ProductNumber').apply(compute_zscore)


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [7]:
features = ['ProductNumber', 'order_month',
            'prev_month_sales', 'prev_week_sales', 'prev_2_month_sales', 'prev_3_month_sales',
            'moving_avg_3m', 'moving_avg_6m', 'moving_avg_12m', 'moving_avg_18m',
            'var_3m', 'var_6m', 'var_12m', 'var_18m',
            'log_var_3m', 'log_var_6m', 'log_var_12m', 'log_var_18m',
            'yoy_growth', 'sales_2023', 'sales_2024'
            ]
target = 'OrderQuantity'

X = product_sales_df[features]
y = product_sales_df[target]

# define categorical and numeric features
categorical_features = ['ProductNumber']
numeric_features = list(set(features) - set(categorical_features))  # All other features are numeric
# ensure ProductNumber is categorical
product_sales_df['ProductNumber'] = product_sales_df['ProductNumber'].astype(str)

# one-hot encoding for categorical features
encoder = OneHotEncoder(handle_unknown='error', sparse_output=False)

# Check/convert 'ProductNumber' to string before fitting OneHotEncoder
X['ProductNumber'] = X['ProductNumber'].astype(str)

# Preprocessing pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='error', sparse_output=False))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)



C:\Users\44777\AppData\Local\Temp\ipykernel_10632\2881775782.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['ProductNumber'] = X['ProductNumber'].astype(str)


In [8]:
X_preprocessed = preprocessor.fit_transform(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

### Random Forest Model

In [ ]:
from model_utils import *

# train the model with features (Random Forest Regressor)
rf_params = find_best_hyperparameters(RandomForestRegressor(), param_grids(RandomForestRegressor().__class__.__name__), X_train, y_train)

Model type: RandomForestRegressor
Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
rf_tuned = RandomForestRegressor(**rf_params)

rf_tuned.fit(X_train, y_train)
rf_y_pred = rf_tuned.predict(y_val)

In [ ]:
print('---------------------- TRAINING PERFORMANCE ----------------------')
evaluate_model(rf_tuned, X_train, y_train)
print('--------------------------------------------')

print('---------------------- TEST PERFORMANCE ----------------------')
evaluate_model(rf_tuned, X_val, y_val)
print('--------------------------------------------')

### Neural Network Model

In [ ]:
mlp_params = find_best_hyperparameters(MLPRegressor(), param_grids(MLPRegressor().__name__), X_train, y_train)

In [ ]:
mlp_tuned = MLPRegressor(**mlp_params)

mlp_tuned.fit(X_train, y_train)
mlp_y_pred = mlp_tuned.predict(y_val)

# Evaluate model using the imported function
print('---------------------- TRAINING PERFORMANCE ----------------------')
train_metrics = evaluate_model(mlp_tuned, X_train, y_train)
print('--------------------------------------------')
print('---------------------- TEST PERFORMANCE ----------------------')
test_metrics = evaluate_model(mlp_tuned, X_val, y_val)
print('--------------------------------------------')

### XGBoost Model

In [ ]:
xgb_params = find_best_hyperparameters(XGBRegressor(), param_grids(XGBRegressor().__name__), X_train, y_train)

In [ ]:
xgb_tuned = XGBRegressor(**xgb_params)

xgb_tuned.fit(X_train, y_train)
xgb_y_pred = xgb_tuned.predict(y_val)

# Evaluate model using the imported function
print('---------------------- TRAINING PERFORMANCE ----------------------')
train_metrics = evaluate_model(xgb_tuned, X_train, y_train)
print('--------------------------------------------')
print('---------------------- TEST PERFORMANCE ----------------------')
test_metrics = evaluate_model(xgb_tuned, X_val, y_val)
print('--------------------------------------------')

In [ ]:
# Create a DetailedOrderDate column using week and weekday information.
# Adjust the format as needed based on your data.
product_sales_df['DetailedOrderDate'] = pd.to_datetime(
    product_sales_df['order_year'].astype(str) +
    product_sales_df['order_week'].astype(str).str.zfill(2) +
    product_sales_df['order_weekday'].astype(str),
    format='%Y%W%w'
)

In [ ]:
# Convert order year and month to datetime format
product_sales_df['OrderDate'] = pd.to_datetime(product_sales_df[['order_year', 'order_month']].astype(str).agg('-'.join, axis=1))

# Sort by OrderDate to maintain time order
product_sales_df = product_sales_df.sort_values(by='OrderDate')

### Plotting Model Results

In [ ]:
# Create subplots (2 rows, 2 columns: one for line plot, one for residual plot)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 12)) 
fig.subplots_adjust(hspace=0.5, wspace=0.4)

# Time Series Comparison
sns.lineplot(x=product_sales_df.loc[y_val.index, 'OrderDate'], y=y_val, label='Actual', ax=axes[0, 0])
sns.lineplot(x=product_sales_df.loc[y_val.index, 'OrderDate'], y=rf_y_pred, label='Predicted', ax=axes[0, 0])
axes[0, 0].set_title(f'Random Forest - Time Series', fontsize=16)
axes[0, 0].set_xlabel('Date', fontsize=14)  
axes[0, 0].set_ylabel('Order Quantity', fontsize=14)

# Residual Plot
residuals = y_val - rf_y_pred
sns.scatterplot(x=rf_y_pred, y=residuals, alpha=0.6, ax=axes[0, 1])
axes[0, 1].axhline(0, color='r', linestyle='--')
axes[0, 1].set_title(f'Random Forest - Residuals', fontsize=16)  
axes[0, 1].set_xlabel('Predicted Values', fontsize=14)  
axes[0, 1].set_ylabel('Scaled Residuals', fontsize=14)

# Actual vs Predicted Scatter Plot
min_val = min(y_val.min(), rf_y_pred.min())
max_val = max(y_val.max(), rf_y_pred.max())
sns.scatterplot(x=y_val, y=rf_y_pred, alpha=0.6, ax=axes[0, 2], label='Predicted')
axes[0, 2].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1)  # Reference line
axes[0, 2].set_title(f'Random Forest - Accuracy', fontsize=16)
axes[0, 2].set_xlabel('Actual Values', fontsize=14)
axes[0, 2].set_ylabel('Predicted Values', fontsize=14)  
axes[0, 2].legend(fontsize=12)

# Adjust x-tick labels after all plots are drawn
plt.setp(axes[0, 0].get_xticklabels(), rotation=45, ha='right', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

# Bar plot: Actual vs. Predicted
axes[0, 0].bar(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    y_val, 
    label='Actual', 
    alpha=0.6
)
axes[0, 0].bar(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    rf_y_pred, 
    label='Predicted', 
    alpha=0.6
)
axes[0, 0].set_xlabel('Order Date')
axes[0, 0].set_ylabel('Order Quantity')
axes[0, 0].set_title(f'{custom_ref} Total Products - Actual vs Predicted (Bar Plot)')
axes[0, 0].set_ylim(0, 1_000_000)
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=45)

# Residual Plot
residuals = y_val - rf_y_pred
axes[0, 1].scatter(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    residuals, 
    alpha=0.6
)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Order Date')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title(f'Total Products - Residual Plot')
axes[0, 1].tick_params(axis='x', rotation=45)

# Pie Chart: Order Quantity Distribution (2024)
order_2024 = product_sales_df[product_sales_df['order_year'] == 2024] \
    .groupby('order_month')['OrderQuantity'].sum()
axes[0, 2].pie(
    order_2024,
    labels=order_2024.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.Paired.colors
)
axes[0, 2].set_title(f'{custom_ref} Order Quantity Distribution (2024)')

# Line Plots: Yearly Trends
for i, year in enumerate([2022, 2023, 2024]):  
    yearly_df = product_sales_df[product_sales_df['order_year'] == year]
    if yearly_df.empty:
        continue

    # Aggregate OrderQuantity per month
    monthly_actual = yearly_df.groupby('order_month')['OrderQuantity'].sum()

    # Convert to Pandas Series with index from product_sales_df
    pred_series = pd.Series(rf_y_pred, index=y_val.index)

    # Group predictions by month
    monthly_predicted = pred_series.groupby(product_sales_df.loc[y_val.index, 'order_month']).sum()

    # Sort for proper plotting
    monthly_actual = monthly_actual.sort_index()
    monthly_predicted = monthly_predicted.reindex(monthly_actual.index)  # Ensure same months

    # Plot
    axes[1, i].plot(
        monthly_actual.index,
        monthly_actual,
        label='Actual',
        alpha=0.6,
        marker='o',
        linestyle='-'
    )
    axes[1, i].plot(
        monthly_predicted.index,
        monthly_predicted,
        label='Predicted',
        alpha=0.6,
        marker='x',
        linestyle='--'
    )
    axes[1, i].set_xlabel('Month')
    axes[1, i].set_ylabel('Order Quantity')
    axes[1, i].set_title(f'{year} Total Products - Actual vs Predicted')
    axes[1, i].set_ylim(0, max(monthly_actual.max(), monthly_predicted.max()) * 1.1)
    axes[1, i].legend()
    axes[1, i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Create subplots (2 rows, 2 columns: one for line plot, one for residual plot)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 12)) 
fig.subplots_adjust(hspace=0.5, wspace=0.4)

# Time Series Comparison
sns.lineplot(x=product_sales_df.loc[y_val.index, 'OrderDate'], y=y_val, label='Actual', ax=axes[0, 0])
sns.lineplot(x=product_sales_df.loc[y_val.index, 'OrderDate'], y=mlp_y_pred, label='Predicted', ax=axes[0, 0])
axes[0, 0].set_title(f'Neural Network (MLP) - Time Series', fontsize=16)
axes[0, 0].set_xlabel('Date', fontsize=14)  
axes[0, 0].set_ylabel('Order Quantity', fontsize=14)

# Residual Plot
residuals = y_val - mlp_y_pred
sns.scatterplot(x=mlp_y_pred, y=residuals, alpha=0.6, ax=axes[0, 1])
axes[0, 1].axhline(0, color='r', linestyle='--')
axes[0, 1].set_title(f'Neural Network (MLP) - Residuals', fontsize=16)  
axes[0, 1].set_xlabel('Predicted Values', fontsize=14)  
axes[0, 1].set_ylabel('Scaled Residuals', fontsize=14)

# Actual vs Predicted Scatter Plot
min_val = min(y_val.min(), mlp_y_pred.min())
max_val = max(y_val.max(), mlp_y_pred.max())
sns.scatterplot(x=y_val, y=mlp_y_pred, alpha=0.6, ax=axes[0, 2], label='Predicted')
axes[0, 2].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1)  # Reference line
axes[0, 2].set_title(f'Neural Network (MLP) - Accuracy', fontsize=16)
axes[0, 2].set_xlabel('Actual Values', fontsize=14)
axes[0, 2].set_ylabel('Predicted Values', fontsize=14)  
axes[0, 2].legend(fontsize=12)

# Adjust x-tick labels after all plots are drawn
plt.setp(axes[0, 0].get_xticklabels(), rotation=45, ha='right', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

# Bar plot: Actual vs. Predicted
axes[0, 0].bar(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    y_val, 
    label='Actual', 
    alpha=0.6
)
axes[0, 0].bar(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    mlp_y_pred, 
    label='Predicted', 
    alpha=0.6
)
axes[0, 0].set_xlabel('Order Date')
axes[0, 0].set_ylabel('Order Quantity')
axes[0, 0].set_title(f'{custom_ref} Total Products - Actual vs Predicted (Bar Plot)')
axes[0, 0].set_ylim(0, 1_000_000)
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=45)

# Residual Plot
residuals = y_val - mlp_y_pred
axes[0, 1].scatter(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    residuals, 
    alpha=0.6
)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Order Date')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title(f'Total Products - Residual Plot')
axes[0, 1].tick_params(axis='x', rotation=45)

# Pie Chart: Order Quantity Distribution (2024)
order_2024 = product_sales_df[product_sales_df['order_year'] == 2024] \
    .groupby('order_month')['OrderQuantity'].sum()
axes[0, 2].pie(
    order_2024,
    labels=order_2024.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.Paired.colors
)
axes[0, 2].set_title(f'{custom_ref} Order Quantity Distribution (2024)')

# Line Plots: Yearly Trends
for i, year in enumerate([2022, 2023, 2024]):  
    yearly_df = product_sales_df[product_sales_df['order_year'] == year]
    if yearly_df.empty:
        continue

    # Aggregate OrderQuantity per month
    monthly_actual = yearly_df.groupby('order_month')['OrderQuantity'].sum()

    # Convert to Pandas Series with index from product_sales_df
    pred_series = pd.Series(mlp_y_pred, index=y_val.index)

    # Group predictions by month
    monthly_predicted = pred_series.groupby(product_sales_df.loc[y_val.index, 'order_month']).sum()

    # Sort for proper plotting
    monthly_actual = monthly_actual.sort_index()
    monthly_predicted = monthly_predicted.reindex(monthly_actual.index)  # Ensure same months

    # Plot
    axes[1, i].plot(
        monthly_actual.index,
        monthly_actual,
        label='Actual',
        alpha=0.6,
        marker='o',
        linestyle='-'
    )
    axes[1, i].plot(
        monthly_predicted.index,
        monthly_predicted,
        label='Predicted',
        alpha=0.6,
        marker='x',
        linestyle='--'
    )
    axes[1, i].set_xlabel('Month')
    axes[1, i].set_ylabel('Order Quantity')
    axes[1, i].set_title(f'{year} Total Products - Actual vs Predicted')
    axes[1, i].set_ylim(0, max(monthly_actual.max(), monthly_predicted.max()) * 1.1)
    axes[1, i].legend()
    axes[1, i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots (2 rows, 2 columns: one for line plot, one for residual plot)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 12)) 
fig.subplots_adjust(hspace=0.5, wspace=0.4)

# Time Series Comparison
sns.lineplot(x=product_sales_df.loc[y_val.index, 'OrderDate'], y=y_val, label='Actual', ax=axes[0, 0])
sns.lineplot(x=product_sales_df.loc[y_val.index, 'OrderDate'], y=xgb_y_pred, label='Predicted', ax=axes[0, 0])
axes[0, 0].set_title(f'XGBoost - Time Series', fontsize=16)
axes[0, 0].set_xlabel('Date', fontsize=14)  
axes[0, 0].set_ylabel('Order Quantity', fontsize=14)

# Residual Plot
residuals = y_val - xgb_y_pred
sns.scatterplot(x=xgb_y_pred, y=residuals, alpha=0.6, ax=axes[0, 1])
axes[0, 1].axhline(0, color='r', linestyle='--')
axes[0, 1].set_title(f'XGBoost - Residuals', fontsize=16)  
axes[0, 1].set_xlabel('Predicted Values', fontsize=14)  
axes[0, 1].set_ylabel('Scaled Residuals', fontsize=14)

# Actual vs Predicted Scatter Plot
min_val = min(y_val.min(), xgb_y_pred.min())
max_val = max(y_val.max(), xgb_y_pred.max())
sns.scatterplot(x=y_val, y=xgb_y_pred, alpha=0.6, ax=axes[0, 2], label='Predicted')
axes[0, 2].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1)  # Reference line
axes[0, 2].set_title(f'XGBoost - Accuracy', fontsize=16)
axes[0, 2].set_xlabel('Actual Values', fontsize=14)
axes[0, 2].set_ylabel('Predicted Values', fontsize=14)  
axes[0, 2].legend(fontsize=12)

# Adjust x-tick labels after all plots are drawn
plt.setp(axes[0, 0].get_xticklabels(), rotation=45, ha='right', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

# Bar plot: Actual vs. Predicted
axes[0, 0].bar(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    y_val, 
    label='Actual', 
    alpha=0.6
)
axes[0, 0].bar(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    xgb_y_pred, 
    label='Predicted', 
    alpha=0.6
)
axes[0, 0].set_xlabel('Order Date')
axes[0, 0].set_ylabel('Order Quantity')
axes[0, 0].set_title(f'{custom_ref} Total Products - Actual vs Predicted (Bar Plot)')
axes[0, 0].set_ylim(0, 1_000_000)
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=45)

# Residual Plot
residuals = y_val - xgb_y_pred
axes[0, 1].scatter(
    product_sales_df.loc[X_val.index, 'OrderDate'], 
    residuals, 
    alpha=0.6
)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Order Date')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title(f'Total Products - Residual Plot')
axes[0, 1].tick_params(axis='x', rotation=45)

# Pie Chart: Order Quantity Distribution (2024)
order_2024 = product_sales_df[product_sales_df['order_year'] == 2024] \
    .groupby('order_month')['OrderQuantity'].sum()
axes[0, 2].pie(
    order_2024,
    labels=order_2024.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.Paired.colors
)
axes[0, 2].set_title(f'{custom_ref} Order Quantity Distribution (2024)')

# Line Plots: Yearly Trends
for i, year in enumerate([2022, 2023, 2024]):  
    yearly_df = product_sales_df[product_sales_df['order_year'] == year]
    if yearly_df.empty:
        continue

    # Aggregate OrderQuantity per month
    monthly_actual = yearly_df.groupby('order_month')['OrderQuantity'].sum()

    # Convert to Pandas Series with index from product_sales_df
    pred_series = pd.Series(xgb_y_pred, index=y_val.index)

    # Group predictions by month
    monthly_predicted = pred_series.groupby(product_sales_df.loc[y_val.index, 'order_month']).sum()

    # Sort for proper plotting
    monthly_actual = monthly_actual.sort_index()
    monthly_predicted = monthly_predicted.reindex(monthly_actual.index)  # Ensure same months

    # Plot
    axes[1, i].plot(
        monthly_actual.index,
        monthly_actual,
        label='Actual',
        alpha=0.6,
        marker='o',
        linestyle='-'
    )
    axes[1, i].plot(
        monthly_predicted.index,
        monthly_predicted,
        label='Predicted',
        alpha=0.6,
        marker='x',
        linestyle='--'
    )
    axes[1, i].set_xlabel('Month')
    axes[1, i].set_ylabel('Order Quantity')
    axes[1, i].set_title(f'{year} Total Products - Actual vs Predicted')
    axes[1, i].set_ylim(0, max(monthly_actual.max(), monthly_predicted.max()) * 1.1)
    axes[1, i].legend()
    axes[1, i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()